# Import

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect(':memory:')

In [3]:
def csv_to_sql(tablename:str):
    df = pd.read_csv(f'{tablename}.csv')
    df.to_sql(tablename, conn, if_exists='replace', index=False)

In [4]:
def show_tables():
    print("stores")
    print(sql("SELECT * FROM stores LIMIT 3"))
    print("products")
    print(sql("SELECT * FROM products LIMIT 3"))
    print("customers")
    print(sql("SELECT * FROM customers LIMIT 3"))
    print("transactions")
    print(sql("SELECT * FROM transactions LIMIT 3"))

In [5]:
csv_to_sql('stores')
csv_to_sql('products')
csv_to_sql('customers')
csv_to_sql('transactions')

In [6]:
sql = lambda x: pd.read_sql_query(x, conn)

# Exploration des Tables et Pré-Analyse

## - Lister les noms des colonnes et le type de données pour chaque table.

In [7]:
sql("PRAGMA table_info(stores);")

,cid,name,type,notnull,dflt_value,pk
0,0,store_id,INTEGER,0,None,0
1,1,store_name,TEXT,0,None,0
2,2,location,TEXT,0,None,0


In [8]:
sql("PRAGMA table_info(products);")

,cid,name,type,notnull,dflt_value,pk
0,0,product_id,INTEGER,0,None,0
1,1,product_name,TEXT,0,None,0
2,2,category,TEXT,0,None,0
3,3,price,REAL,0,None,0


In [9]:
sql("PRAGMA table_info(customers);")

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,INTEGER,0,None,0
1,1,customer_name,TEXT,0,None,0
2,2,age,INTEGER,0,None,0
3,3,gender,TEXT,0,None,0


In [10]:
sql("PRAGMA table_info(transactions);")

,cid,name,type,notnull,dflt_value,pk
0,0,transaction_id,INTEGER,0,None,0
1,1,customer_id,INTEGER,0,None,0
2,2,product_id,INTEGER,0,None,0
3,3,store_id,INTEGER,0,None,0
4,4,date,TEXT,0,None,0
5,5,amount,REAL,0,None,0


## Vérifier s'il y a des doublons dans la table transactions sur la colonne transaction_id.

In [11]:
# calcul de l'occurence des valeurs

In [12]:
sql('SELECT transaction_id, COUNT(*) as count FROM transactions GROUP BY transaction_id')

,transaction_id,count
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
995,996,1
996,997,1
997,998,1
998,999,1


In [13]:
# filtre des valeurs occurents plus d'une fois

In [14]:
sql('SELECT transaction_id, COUNT(*) as count FROM transactions GROUP BY transaction_id HAVING count > 1')

,transaction_id,count


## Identifier les clés primaires et les clés étrangères de chaque table.

In [15]:
sql("PRAGMA table_info(stores);")

,cid,name,type,notnull,dflt_value,pk
0,0,store_id,INTEGER,0,None,0
1,1,store_name,TEXT,0,None,0
2,2,location,TEXT,0,None,0


In [16]:
sql("PRAGMA foreign_key_list('stores');")

,id,seq,table,from,to,on_update,on_delete,match


In [17]:
sql("PRAGMA table_info(products);")

,cid,name,type,notnull,dflt_value,pk
0,0,product_id,INTEGER,0,None,0
1,1,product_name,TEXT,0,None,0
2,2,category,TEXT,0,None,0
3,3,price,REAL,0,None,0


In [18]:
sql("PRAGMA foreign_key_list('products');")

,id,seq,table,from,to,on_update,on_delete,match


In [19]:
sql("PRAGMA table_info(customers);")

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,INTEGER,0,None,0
1,1,customer_name,TEXT,0,None,0
2,2,age,INTEGER,0,None,0
3,3,gender,TEXT,0,None,0


In [20]:
sql("PRAGMA foreign_key_list('customers');")

,id,seq,table,from,to,on_update,on_delete,match


In [21]:
sql("PRAGMA table_info(transactions);")

,cid,name,type,notnull,dflt_value,pk
0,0,transaction_id,INTEGER,0,None,0
1,1,customer_id,INTEGER,0,None,0
2,2,product_id,INTEGER,0,None,0
3,3,store_id,INTEGER,0,None,0
4,4,date,TEXT,0,None,0
5,5,amount,REAL,0,None,0


In [22]:
sql("PRAGMA foreign_key_list('transactions');")

,id,seq,table,from,to,on_update,on_delete,match


## Vérifier si la colonne date dans la table transactions est stockée dans un format date, sinon, la convertir.

In [23]:
sql("PRAGMA table_info(transactions);")

,cid,name,type,notnull,dflt_value,pk
0,0,transaction_id,INTEGER,0,None,0
1,1,customer_id,INTEGER,0,None,0
2,2,product_id,INTEGER,0,None,0
3,3,store_id,INTEGER,0,None,0
4,4,date,TEXT,0,None,0
5,5,amount,REAL,0,None,0


In [24]:
# JPEUT PAAAAAAAAAAAAAAAAAAAAAS

## Vérifier s'il existe des valeurs nulles dans les colonnes customer_id, product_id, store_id des transactions.

In [25]:
sql("SELECT COUNT(*) as count_null_values FROM transactions WHERE customer_id + product_id + store_id IS NULL;")

,count_null_values
0,0


# Requêtes de Base

## Lister toutes les transactions effectuées dans les magasin de New York.

In [26]:
sql('SELECT * FROM stores LEFT JOIN transactions ON stores.store_id = transactions.store_id WHERE location = "New York"')

,store_id,store_name,location,transaction_id,customer_id,product_id,store_id,date,amount
0,1,Store 1,New York,7,146,78,1,2023-01-07,48.99
1,1,Store 1,New York,12,500,81,1,2023-01-12,11.31
2,1,Store 1,New York,29,70,52,1,2023-01-29,290.12
3,1,Store 1,New York,51,400,62,1,2023-02-20,733.48
4,1,Store 1,New York,76,452,29,1,2023-03-17,277.07
...,...,...,...,...,...,...,...,...,...
396,20,Store 20,New York,937,104,88,20,2025-07-25,905.52
397,20,Store 20,New York,941,370,74,20,2025-07-29,308.36
398,20,Store 20,New York,960,497,2,20,2025-08-17,127.38
399,20,Store 20,New York,978,292,95,20,2025-09-04,218.93


In [27]:
# Il y a 401 eu transactions a new york

## Récupérer les noms des produits achetés par le customer_id 35.

In [28]:
# produits achetés par le customer 35

In [29]:
sql('SELECT * FROM transactions WHERE customer_id = 35')

,transaction_id,customer_id,product_id,store_id,date,amount
0,1,35,8,12,2023-01-01,379.09
1,483,35,96,3,2024-04-27,959.61


In [30]:
# détails des produits achetés par le customer 35

In [31]:
sql('SELECT * FROM transactions LEFT JOIN products ON transactions.product_id = products.product_id WHERE customer_id = 35')

,transaction_id,customer_id,product_id,store_id,date,amount,product_id,product_name,category,price
0,1,35,8,12,2023-01-01,379.09,8,Product 8,Books,388.96
1,483,35,96,3,2024-04-27,959.61,96,Product 96,Furniture,112.24


## Lister les différents types de catégories de produits disponibles.

In [32]:
sql("SELECT DISTINCT(category) FROM products")

,category
0,Books
1,Electronics
2,Furniture
3,Clothing


## Trouver le montant total des transactions pour le store_id 12.

In [33]:
sql("SELECT COUNT(*) as total_transactions_store_12 FROM transactions WHERE store_id = 12")

,total_transactions_store_12
0,49


## Lister tous les clients de sexe féminin.

In [34]:
sql('SELECT * FROM CUSTOMERS WHERE gender = "Female"')

,customer_id,customer_name,age,gender
0,2,Customer 2,40,Female
1,7,Customer 7,55,Female
2,9,Customer 9,51,Female
3,10,Customer 10,22,Female
4,11,Customer 11,69,Female
...,...,...,...,...
226,490,Customer 490,53,Female
227,494,Customer 494,35,Female
228,496,Customer 496,59,Female
229,497,Customer 497,50,Female


# Fonctions d’Agrégation et GROUP BY

## Calculer la moyenne des montants des transactions par magasin.

In [35]:
sql('SELECT store_id, AVG(amount) as mean_amount_by_stores  FROM transactions GROUP BY store_id')

,store_id,mean_amount_by_stores
0,1,532.130000
1,2,496.529592
2,3,481.485789
3,4,563.986364
4,5,614.618200
5,6,465.471190
6,7,503.759412
7,8,461.190196
8,9,465.133200
9,10,504.817115


## Trouver le magasin qui a réalisé le chiffre d'affaires le plus élevé.

In [36]:
sql('SELECT store_id, SUM(amount) as sum_amount_by_stores  FROM transactions GROUP BY store_id ORDER BY sum_amount_by_stores DESC LIMIT 1')

,store_id,sum_amount_by_stores
0,5,30730.91


## Calculer le montant total dépensé par chaque client.

In [37]:
sql("SELECT customer_id, SUM(amount) total_spent_by_customers FROM transactions GROUP BY customer_id")

,customer_id,total_spent_by_customers
0,1,2528.13
1,2,2108.46
2,3,427.58
3,4,1447.50
4,5,574.77
...,...,...
434,496,2437.91
435,497,1821.58
436,498,1672.92
437,499,486.39


## Trouver le produit le plus vendu en termes de quantité.

In [38]:
sql("SELECT product_id, COUNT(*) as total_sold FROM transactions GROUP BY product_id ORDER BY total_sold DESC LIMIT 1")

,product_id,total_sold
0,45,20


## Lister les catégories de produits et le montant total des ventes par catégorie.

In [39]:
sql("SELECT category, SUM(amount) total_amount FROM transactions t LEFT JOIN products p ON t.product_id = p.product_id GROUP BY category")

,category,total_amount
0,Books,151828.24
1,Clothing,108614.35
2,Electronics,125958.34
3,Furniture,127514.47


# Clauses HAVING vs WHERE

## Lister les clients qui ont effectué plus de 5 transactions.

In [40]:
sql("SELECT customer_id, COUNT(*) total_transactions FROM transactions GROUP BY customer_id HAVING total_transactions > 5")

,customer_id,total_transactions
0,61,6
1,141,6
2,277,6


## Trouver les produits qui ont généré plus de 1000 en ventes totales.

In [41]:
sql("SELECT product_id, SUM(amount) total_amount FROM transactions GROUP BY product_id HAVING total_amount > 1000")

,product_id,total_amount
0,1,4334.66
1,2,5735.32
2,3,5909.48
3,4,3590.36
4,5,4182.23
...,...,...
95,96,6980.49
96,97,6947.89
97,98,4940.70
98,99,5780.27


## Lister les magasins ayant une moyenne de transaction supérieure à 200.

In [42]:
sql("SELECT store_id, AVG(amount) average_amount FROM transactions GROUP BY store_id HAVING average_amount > 200")

,store_id,average_amount
0,1,532.130000
1,2,496.529592
2,3,481.485789
3,4,563.986364
4,5,614.618200
5,6,465.471190
6,7,503.759412
7,8,461.190196
8,9,465.133200
9,10,504.817115


## Récupérer les produits appartenant à la catégorie Books et ayant généré plus de 500 en ventes.

In [43]:
sql("""SELECT p.product_id, category, SUM(amount) sum_amount 
        FROM transactions t 
        LEFT JOIN products p 
        ON t.product_id = p.product_id
        WHERE category = 'Books'
        GROUP BY p.product_id
        HAVING sum_amount > 500
        """)

,product_id,category,sum_amount
0,1,Books,4334.66
1,2,Books,5735.32
2,3,Books,5909.48
3,6,Books,3147.49
4,8,Books,4192.44
5,9,Books,7749.04
6,14,Books,7558.51
7,20,Books,5097.30
8,31,Books,3538.25
9,33,Books,6547.09


## Trouver les clients ayant dépensé plus de 1000 au total.

In [44]:
sql("SELECT customer_id, SUM(amount) sum_amount FROM transactions GROUP BY customer_id HAVING sum_amount > 1000")

,customer_id,sum_amount
0,1,2528.13
1,2,2108.46
2,4,1447.50
3,6,1934.12
4,7,1303.84
...,...,...
217,494,2268.27
218,495,1814.12
219,496,2437.91
220,497,1821.58


# Fonctions de Manipulation de Chaînes et Dates

## Extraire l’année des dates de transactions et compter le nombre de transactions par année.

In [45]:
sql("SELECT strftime('%Y', date) year, COUNT(*) total_transactions FROM transactions GROUP BY year")

,year,total_transactions
0,2023,365
1,2024,366
2,2025,269


## Concaténer le nom du client avec son âge.

In [46]:
sql('SELECT (customer_name || " " || age) customer_age FROM customers')

,customer_age
0,Customer 1 25
1,Customer 2 40
2,Customer 3 30
3,Customer 4 33
4,Customer 5 43
...,...
495,Customer 496 59
496,Customer 497 50
497,Customer 498 51
498,Customer 499 35


## Remplacer tous les espaces par des underscores dans les noms des produits.

In [47]:
sql("SELECT REPLACE(product_name, ' ', '_') product_name FROM products")

,product_name
0,Product_1
1,Product_2
2,Product_3
3,Product_4
4,Product_5
...,...
95,Product_96
96,Product_97
97,Product_98
98,Product_99


## Transformer en majuscules les noms des magasins.

In [48]:
sql("SELECT UPPER(store_name) store_name FROM stores")

,store_name
0,STORE 1
1,STORE 2
2,STORE 3
3,STORE 4
4,STORE 5
5,STORE 6
6,STORE 7
7,STORE 8
8,STORE 9
9,STORE 10


# Joins

## Joindre la table transactions avec stores et afficher le nom du magasin pour chaque transaction.

In [49]:
sql("SELECT store_name, t.* FROM transactions t LEFT JOIN stores s ON t.store_id = s.store_id")

,store_name,transaction_id,customer_id,product_id,store_id,date,amount
0,Store 12,1,35,8,12,2023-01-01,379.09
1,Store 16,2,236,34,16,2023-01-02,18.48
2,Store 20,3,334,50,20,2023-01-03,783.51
3,Store 20,4,325,46,20,2023-01-04,333.82
4,Store 6,5,332,89,6,2023-01-05,329.74
...,...,...,...,...,...,...,...
995,Store 8,996,271,62,8,2025-09-22,268.09
996,Store 4,997,380,9,4,2025-09-23,829.86
997,Store 14,998,46,22,14,2025-09-24,603.12
998,Store 8,999,2,85,8,2025-09-25,882.43


## Joindre les tables transactions et products pour obtenir le nom du produit acheté dans chaque transaction.

In [50]:
sql("SELECT product_name , t.* FROM transactions t LEFT JOIN products s ON t.product_id = s.product_id")

,product_name,transaction_id,customer_id,product_id,store_id,date,amount
0,Product 8,1,35,8,12,2023-01-01,379.09
1,Product 34,2,236,34,16,2023-01-02,18.48
2,Product 50,3,334,50,20,2023-01-03,783.51
3,Product 46,4,325,46,20,2023-01-04,333.82
4,Product 89,5,332,89,6,2023-01-05,329.74
...,...,...,...,...,...,...,...
995,Product 62,996,271,62,8,2025-09-22,268.09
996,Product 9,997,380,9,4,2025-09-23,829.86
997,Product 22,998,46,22,14,2025-09-24,603.12
998,Product 85,999,2,85,8,2025-09-25,882.43


## Faire une jointure entre transactions, products, et stores pour afficher le produit acheté, le magasin et le montant.

In [51]:
sql("""SELECT * 
    FROM transactions t
    LEFT JOIN products p ON t.product_id = p.product_id
    LEFT join stores s ON t.store_id = s.store_id
""")

,transaction_id,customer_id,product_id,store_id,date,amount,product_id,product_name,category,price,store_id,store_name,location
0,1,35,8,12,2023-01-01,379.09,8,Product 8,Books,388.96,12,Store 12,Los Angeles
1,2,236,34,16,2023-01-02,18.48,34,Product 34,Electronics,392.02,16,Store 16,New York
2,3,334,50,20,2023-01-03,783.51,50,Product 50,Books,291.19,20,Store 20,New York
3,4,325,46,20,2023-01-04,333.82,46,Product 46,Clothing,240.86,20,Store 20,New York
4,5,332,89,6,2023-01-05,329.74,89,Product 89,Clothing,125.60,6,Store 6,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,271,62,8,2025-09-22,268.09,62,Product 62,Books,183.73,8,Store 8,Houston
996,997,380,9,4,2025-09-23,829.86,9,Product 9,Books,419.09,4,Store 4,New York
997,998,46,22,14,2025-09-24,603.12,22,Product 22,Clothing,203.65,14,Store 14,New York
998,999,2,85,8,2025-09-25,882.43,85,Product 85,Furniture,204.37,8,Store 8,Houston


## Lister les transactions pour les clients de plus de 30 ans.

In [52]:
sql("SELECT * FROM transactions t LEFT JOIN customers c ON t.customer_id = c.customer_id WHERE age > 30")

,transaction_id,customer_id,product_id,store_id,date,amount,customer_id,customer_name,age,gender
0,418,2,17,20,2024-02-22,556.39,2,Customer 2,40,Female
1,666,2,100,3,2024-10-27,669.64,2,Customer 2,40,Female
2,999,2,85,8,2025-09-25,882.43,2,Customer 2,40,Female
3,446,4,93,3,2024-03-21,171.69,4,Customer 4,33,Male
4,568,4,99,4,2024-07-21,847.98,4,Customer 4,33,Male
...,...,...,...,...,...,...,...,...,...,...
781,345,498,60,15,2023-12-11,463.53,498,Customer 498,51,Male
782,958,498,98,4,2025-08-15,283.84,498,Customer 498,51,Male
783,127,499,34,10,2023-05-07,486.39,499,Customer 499,35,Male
784,12,500,81,1,2023-01-12,11.31,500,Customer 500,69,Female


## Lister tous les produits achetés par les clients de sexe masculin.

In [53]:
sql("SELECT * FROM transactions t LEFT JOIN customers c ON t.customer_id = c.customer_id WHERE gender = 'Male'")

,transaction_id,customer_id,product_id,store_id,date,amount,customer_id,customer_name,age,gender
0,32,1,83,3,2023-02-01,377.15,1,Customer 1,25,Male
1,761,1,88,1,2025-01-30,886.60,1,Customer 1,25,Male
2,871,1,35,9,2025-05-20,649.32,1,Customer 1,25,Male
3,965,1,89,5,2025-08-22,615.06,1,Customer 1,25,Male
4,152,3,24,9,2023-06-01,284.59,3,Customer 3,30,Male
...,...,...,...,...,...,...,...,...,...,...
545,370,495,33,6,2024-01-05,106.43,495,Customer 495,46,Male
546,186,498,74,7,2023-07-05,925.55,498,Customer 498,51,Male
547,345,498,60,15,2023-12-11,463.53,498,Customer 498,51,Male
548,958,498,98,4,2025-08-15,283.84,498,Customer 498,51,Male


# Sous-requêtes (Subqueries) et CTE

## Trouver les clients qui ont effectué une transaction pour un montant supérieur à la moyenne des transactions.

In [54]:
sql("""
    SELECT 
        t.customer_id, 
        amount
    FROM transactions t
    LEFT JOIN customers c ON t.customer_id = c.customer_id
    WHERE amount > (
        SELECT AVG(amount) avg_amount FROM transactions
    )
""")

,customer_id,amount
0,334,783.51
1,379,811.79
2,42,515.03
3,28,826.28
4,453,888.98
...,...,...
497,351,934.32
498,189,724.74
499,380,829.86
500,46,603.12


## Lister les produits ayant un prix supérieur au prix moyen de leur catégorie.

In [55]:
sql("""
    WITH a as (
        SELECT 
            category, AVG(price) avg_price
        FROM 
            products 
        GROUP BY 
            category
    )
    SELECT 
        DISTINCT 
        product_name, 
        p.category, 
        price, 
        avg_price
    FROM 
        transactions t
    LEFT JOIN products p 
        ON t.product_id = p.product_id
    LEFT JOIN a 
        ON a.category = p.category
    WHERE 
        price > avg_price
    """)

,product_name,category,price,avg_price
0,Product 2,Books,291.83,256.005714
1,Product 3,Books,295.06,256.005714
2,Product 8,Books,388.96,256.005714
3,Product 9,Books,419.09,256.005714
4,Product 14,Books,309.15,256.005714
5,Product 20,Books,400.85,256.005714
6,Product 31,Books,488.03,256.005714
7,Product 42,Books,361.28,256.005714
8,Product 47,Books,265.57,256.005714
9,Product 50,Books,291.19,256.005714


## Utiliser un CTE pour calculer le montant total des transactions par client, puis filtrer pour trouver ceux ayant dépensé plus de 500.

In [56]:
sql("SELECT customer_id, SUM(amount) total_spent FROM transactions GROUP BY customer_id HAVING total_spent > 500")

,customer_id,total_spent
0,1,2528.13
1,2,2108.46
2,4,1447.50
3,5,574.77
4,6,1934.12
...,...,...
338,494,2268.27
339,495,1814.12
340,496,2437.91
341,497,1821.58


In [57]:
sql("""
    WITH truc as (
        SELECT customer_id, SUM(amount) total_spent FROM transactions GROUP BY customer_id
    )
    SELECT c.customer_id, customer_name
    FROM customers c
    LEFT JOIN truc ON truc.customer_id = c.customer_id
    WHERE total_spent > 500
""")

,customer_id,customer_name
0,1,Customer 1
1,2,Customer 2
2,4,Customer 4
3,5,Customer 5
4,6,Customer 6
...,...,...
338,494,Customer 494
339,495,Customer 495
340,496,Customer 496
341,497,Customer 497


## Lister les produits qui ont été achetés plus de 10 fois et appartenant à la catégorie Electronics.

In [58]:
sql("""
    WITH elec_categ as (
        SELECT 
            product_id, 
            category 
        FROM 
            products 
        WHERE 
            category = "Electronics"
    )
    SELECT 
        t.product_id, 
        COUNT(*) total_sold, 
        p.category
    FROM 
        transactions t
    LEFT JOIN 
        products p ON p.product_id = t.product_id
    LEFT JOIN 
        elec_categ e ON e.product_id = p.product_id
    GROUP BY 
        t.product_id
    HAVING 
        total_sold > 10 AND 
        t.product_id = e.product_id
""")

,product_id,total_sold,category
0,7,13,Electronics
1,18,11,Electronics
2,28,11,Electronics
3,34,14,Electronics
4,35,16,Electronics
5,48,16,Electronics
6,51,13,Electronics
7,58,11,Electronics
8,74,17,Electronics


# Fonctions Fenêtre (Window Functions)

## Attribuer un rang aux transactions en fonction du montant dans chaque magasin.

In [59]:
sql("""
    SELECT transaction_id, store_id, amount, RANK() OVER (PARTITION BY store_id ORDER BY amount DESC) ranking
    FROM transactions t
""")

,transaction_id,store_id,amount,ranking
0,722,1,994.80,1
1,415,1,975.16,2
2,667,1,965.60,3
3,203,1,936.11,4
4,340,1,924.09,5
...,...,...,...,...
995,452,20,95.91,52
996,584,20,69.02,53
997,598,20,49.71,54
998,715,20,42.45,55


## Calculer la somme cumulée du montant des transactions pour chaque client.

In [60]:
sql("""
    SELECT DISTINCT customer_id, SUM(amount) OVER (PARTITION BY customer_id) total_spent
    FROM transactions t
    ORDER BY total_spent DESC
""")

,customer_id,total_spent
0,141,5087.76
1,136,4087.82
2,476,3768.81
3,61,3724.36
4,440,3441.46
...,...,...
434,8,28.44
435,129,27.76
436,382,27.43
437,148,24.44


## Attribuer un numéro de rang aux magasins basé sur le chiffre d'affaires total.

In [61]:
sql("""
    SELECT
        store_id, 
        SUM(amount) total_sales, 
        RANK() OVER (ORDER BY SUM(amount) DESC) ranking
    FROM 
        transactions
    GROUP BY 
        store_id
""")

,store_id,total_sales,ranking
0,5,30730.91,1
1,17,29533.06,2
2,19,27947.47,3
3,13,27881.63,4
4,20,27781.14,5
5,11,27649.98,6
6,3,27444.69,7
7,15,26822.47,8
8,12,26796.29,9
9,10,26250.49,10


## Calculer le pourcentage de contribution de chaque transaction au total des ventes du magasin.

In [62]:
sql("""
    SELECT
        transaction_id,
        amount,
        store_id,
        amount * 100.0 / SUM(amount) OVER (PARTITION BY store_id) contribution_percentage,
        SUM(amount) OVER (PARTITION BY store_id) total_store_sales
    FROM
        transactions
""")

,transaction_id,amount,store_id,contribution_percentage,total_store_sales
0,7,48.99,1,0.214102,22881.59
1,12,11.31,1,0.049428,22881.59
2,29,290.12,1,1.267919,22881.59
3,51,733.48,1,3.205546,22881.59
4,76,277.07,1,1.210886,22881.59
...,...,...,...,...,...
995,937,905.52,20,3.259477,27781.14
996,941,308.36,20,1.109962,27781.14
997,960,127.38,20,0.458513,27781.14
998,978,218.93,20,0.788053,27781.14


pour vérifier

In [63]:
sql("""
    WITH store_1 as (
    SELECT
        transaction_id,
        amount,
        store_id,
        amount * 100.0 / SUM(amount) OVER (PARTITION BY store_id) contribution_percentage,
        SUM(amount) OVER (PARTITION BY store_id) total_store_sales
    FROM
        transactions
    WHERE store_id = 1
    )
    SELECT
        SUM(contribution_percentage) sum_contribution_percentage_store_1
    FROM
        store_1
""")

,sum_contribution_percentage_store_1
0,100.0


In [64]:
show_tables()

stores
   store_id store_name     location
0         1    Store 1     New York
1         2    Store 2  Los Angeles
2         3    Store 3     New York
products
   product_id product_name category   price
0           1    Product 1    Books   14.02
1           2    Product 2    Books  291.83
2           3    Product 3    Books  295.06
customers
   customer_id customer_name  age  gender
0            1    Customer 1   25    Male
1            2    Customer 2   40  Female
2            3    Customer 3   30    Male
transactions
   transaction_id  customer_id  product_id  store_id        date  amount
0               1           35           8        12  2023-01-01  379.09
1               2          236          34        16  2023-01-02   18.48
2               3          334          50        20  2023-01-03  783.51
